In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Inspiration: 
https://www.kaggle.com/roshanchoudhary/forest-cover-walkthrough-in-python-knn-96-51<br>
https://www.kaggle.com/sharmasanthosh/exploratory-study-on-feature-selection

## Data Dictionary

* Elevation = Elevation (altitude) in meters
* Aspect = Aspect in degrees azimuth
* Slope = Slope in degrees
* Horizontal_Distance_To_Hydrology = Horizontal distance to nearest surface water features
* Vertical_Distance_To_Hydrology = Vertical distance to nearest surface water features
* Horizontal_Distance_To_Roadways = Horizontal distance to nearest roadway
* Hillshade_9am = Hill shade index at 9am, summer solstice. Value out of 255
* Hillshade_Noon = Hill shade index at noon, summer solstice. Value out of 255
* Hillshade_3pm = Hill shade index at 3pm, summer solstice. Value out of 255
* Horizontal_Distance_To_Fire_Point = sHorizontal distance to nearest wildfire ignition points
* Wilderness_Area1 = Rawah Wilderness Area
* Wilderness_Area2 = Neota Wilderness Area
* Wilderness_Area3 = Comanche Peak Wilderness Area
* Wilderness_Area4 = Cache la Poudre Wilderness Area
* Soil types 1-40

### Tree species:
1. Spruce/Fir
2. Lodgepole Pine
3. Ponderosa Pine
4. Cottonwood/Willow
5. Aspen
6. Douglas-fir
7. Krummholz

### The problem:
Based on the given data, try to find some patterns in the feature space which influence the result. The result (label) is the forest cover itself, which refers to tree species. With the above features we have to try to find some algorithms that can help us predict the tree species based on environmental/climate parameters on a given plot.
### Real world application:
This prediction can be used for afforestation or reforestation decisions when it comes to finding the right species to plant.
### Approach
The data should be analyzed and statistically described. Then the useful features have to be selected.
This is a multi-class classification problem. We have 7 labels and we have to predict which one applies to given parameters.
To solve classification problems, we can use machine learning algorithms.

### Reading the csv files as a dataframe

In [ ]:
df = pd.read_csv('../input/forest-cover-type-dataset/covtype.csv')

### Checking the dimension of the dataframe

In [ ]:
print("There are {} records and {} features in the dataset".format(df.shape[0], df.shape[1]))

### Checking data types by column

In [ ]:
df.info()

### We can see that all columns contain only integers and there are only non-null values
### We can also visualize the missing values by column if any:

In [ ]:
import missingno as msno
msno.matrix(df)

### We can see again, the dataset is very clear, no missing values

### We want to see all columns when seeing results, so we need to set this option (*Pandas only shows some columns from the beginning and from the end part of the dataframe by default*)

In [ ]:
pd.set_option('display.max_columns', None)

### Displaying the first 5 rows of the dataframe just to have an overview

In [ ]:
df.head()

### Displaying the main statistical metrics for each column

In [ ]:
df.describe()

### Some interesting facts:
* mean elevation is ~2959 m, minimum is 1859 m, maximum is 3858 m
* mean slope is 14°

### Take a look at the correlation between variables

In [ ]:
sb.set(style="white")
df_corr = df.corr()
mask_train = np.triu(np.ones_like(df_corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(20, 20))
cmap_train = sb.diverging_palette(220, 10, as_cmap=True)
sb.heatmap(df_corr, mask=mask_train, cmap=cmap_train, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Check skewness

In [ ]:
print(df.skew())

In [ ]:
df['Cover_Type'].value_counts()

### We'll look at skewness visually as well, but first we need to separate the continuous variables from the dataset

In [ ]:
cont_df = df[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology','Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways',
              'Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Horizontal_Distance_To_Fire_Points']]

In [ ]:
cont_df.head(5)

In [ ]:
sb.set_style("darkgrid")
for i, col in enumerate(cont_df.columns):
    plt.figure(i)
    t = sb.distplot(cont_df[col],color="g",label="Skewness: {0:.2f}".format(cont_df[col].skew()))
    t.legend()

### On the next visuals we can see not just the quartiles and median but also the probability density of the data at different categories.

In [ ]:
df['Cover_Type']=df['Cover_Type'].astype('category')

for i, col in enumerate(cont_df.columns):
    plt.figure(i,figsize=(8,4))
    sb.violinplot(x=df['Cover_Type'], y=col, data=df, palette="mako")

In [ ]:
cols = df.columns

#number of rows=r, number of columns=c
r,c = df.shape

#Create a new dataframe with r rows, one column for each encoded category, and target
newdf = pd.DataFrame(index=np.arange(0, r),columns=['Wilderness_Area','Soil_Type','Cover_Type'])

#Make an entry in 'newdf' for each r as category_id, target value
for x in range(0,r):
    w=0;
    s=0;
    
    # Category1
    for y in range(10,14):
        if (df.iloc[x,y] == 1):
            w=y-9  #category class
            break
            
    # Category2       
    for z in range(14,54):
        if (df.iloc[x,z] == 1):
            s=z-13 #category class
            break
    #Make an entry in 'data' for each r as category_id, target value        
    newdf.iloc[x]=[w,s,df.iloc[x,c-1]]

#Category 1:
sb.countplot(x="Wilderness_Area", hue="Cover_Type", data=newdf, palette="viridis")
plt.show()

#Category 2:
plt.rc("figure", figsize=(25, 10))
sb.countplot(x="Soil_Type", hue="Cover_Type", data=newdf, palette="Spectral")
plt.show()

### Let's look at on the correlations between features again, now only with regards to continuous features

In [ ]:
cont_df.corr()

In [ ]:
plt.figure(figsize=(15,8))
sb.heatmap(cont_df.corr(),cmap='viridis',annot=True)

In [ ]:
g = sb.PairGrid(cont_df)
g.map(sb.scatterplot, color="green")

### Let's divide the dataset to features and label

In [ ]:
X=df.loc[:,'Elevation':'Soil_Type40']
y=df['Cover_Type']

### Let's remove the features with low variance

In [ ]:
colstodrop=['Hillshade_3pm','Soil_Type7','Soil_Type8','Soil_Type14','Soil_Type15',
     'Soil_Type21','Soil_Type25','Soil_Type28','Soil_Type36','Soil_Type37']

In [ ]:
X.drop(colstodrop, axis=1, inplace=True)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

### Split the data into train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=42)
X_train_reduced = X_train[:5000]
y_train_reduced = y_train[:5000]

### Making pipeline including scaling, grid search and cross validation for each model

pipe = make_pipeline(StandardScaler(), SVC())  
param_grid = {'svc__C': [1e3, 1e4, 1e5],
              'svc__gamma': [0.001, 0.01, 0.1]}
gs = GridSearchCV(pipe, param_grid, cv=5)
gs.fit(X_train[:2000], y_train[:2000])

gs.best_params_

gs.best_score_

In [ ]:
clf_svc = make_pipeline(StandardScaler(),SVC(kernel='rbf',C=1000,gamma=0.001))

In [ ]:
clf_svc.fit(X_train_reduced, y_train_reduced)

In [ ]:
y_pred = clf_svc.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
target_names = ['Spruce/Fir', 'Lodgepole Pine', 'Ponderosa Pine','Cottonwood/Willow','Aspen','Douglas-fir','Krummholz']
classification_report(y_test, y_pred, target_names=target_names)

In [ ]:
clf_svc.score(X_test,y_test)

In [ ]:
n = list(range(1,31))

In [ ]:
pipe = make_pipeline(StandardScaler(), KNeighborsClassifier())
param_grid = {'kneighborsclassifier__n_neighbors': n} 
gs = GridSearchCV(pipe, param_grid, cv=5) 
gs.fit(X_train[:20000], y_train[:20000])
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
clf_knn = make_pipeline(StandardScaler(),KNeighborsClassifier(n_neighbors=5))
clf_knn.fit(X_train_reduced, y_train_reduced)

In [ ]:
y_pred = clf_knn.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
classification_report(y_test, y_pred, target_names=target_names)

In [ ]:
clf_knn.score(X_test,y_test)

In [ ]:
clf_nb = make_pipeline(StandardScaler(), GaussianNB())
clf_nb.fit(X_train_reduced, y_train_reduced)


In [ ]:
y_pred = clf_knn.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
classification_report(y_test, y_pred, target_names=target_names)

In [ ]:
clf_nb.score(X_test,y_test)

In [ ]:
pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier())
param_grid = { 'decisiontreeclassifier__max_depth': list(range(3,16)), 'decisiontreeclassifier__max_leaf_nodes': list(range(2, 100)) } 
gs = GridSearchCV(pipe, param_grid, cv=5) 
gs.fit(X_train[:20000], y_train[:20000])
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
clf_tree = make_pipeline(StandardScaler(), DecisionTreeClassifier(max_depth=10,max_leaf_nodes=99))
clf_tree.fit(X_train_reduced, y_train_reduced)
y_pred = clf_knn.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
clf_tree.score(X_test,y_test)

In [ ]:
clf_rf = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=10))
clf_rf.fit(X_train[:100000], y_train[:100000])
y_pred = clf_rf.predict(X_test)
clf_rf.score(X_test, y_test)

In [ ]:
scores = cross_val_score(clf_rf, X_train, y_train, cv=5)
scores